In [2]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import mtalg
from financepy.utils.math import N, n_vect

####################################################################
# FINANCEPY BETA Version 0.310 - This build:  25 Aug 2023 at 16:17 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Задача 1 ##

### Постановка задачи
$
\text{Даны X и Y - cлучайные величины с нормальным распределением: } X \sim \mathcal{N}(e_1, q_1^2) \text{ и } Y \sim \mathcal{N}(e_2, q_2^2) \text{, где corr}(X, Y) = p.
$
$
\text{Необходимо найти такие константы a и b, минимизирующие } \text{Var}[S] \text{, где } S = aX + bY \text{, такие что } a + b = 1, 0 \leq a \leq 1, 0 \leq b \leq 1
$

### Аналитическое решение
$\text{По условию, } \text{cov}(X,Y) = p q_1 q_2 \\$
$\text{Воспользуемся свойствами дисперсии и сформулируем задачу формально: } \\ \text{Min } \text{Var}[S] = a^2 q_1^2 + b^2 q_2^2 + 2abpq_1q_2 \\ \text{при ограничениях: } a + b = 1, 0 \leq a \leq 1, 0 \leq b \leq 1 \\$

$\text{Это задача квадратичного программирования c ограничениями. }$
$\text{В общем случае, такие задачи не имеют аналитического решения.}$ $\text{Попробуем исследовать частное решение, применив метод множителей Лагранжа: } \\ $

$  \text{Создадим функцию Лангража: }L(a, b, \lambda) = a^2 q_1^2 + b^2 q_2^2 + 2abpq_1q_2 + \lambda(1 - a - b)$

$ \text{Найдем частные производные для } a, b, \lambda\ \text{ и приравняем их к нулю:} \\$
$ \frac{\partial L}{\partial a} = 2aq_1^2 + 2bpq_1q_2 - \lambda = 0 \\ $
$ \frac{\partial L}{\partial b} = 2bq_2^2 + 2apq_1q_2 - \lambda = 0 \\ $
$ \frac{\partial L}{\partial \lambda} = 1 - a - b = 0 $
$\text{} \\$
$\text{Решив эту систему уравнений относительно a и b,}$
$\text{мы получим аналитическое решение. Выразим a и b через } \lambda \text{:}\\$
$ a = \frac{\lambda - 2bpq_1q_2}{2q_1^2} \\$
$ b = \frac{\lambda - 2apq_1q_2}{2q_2^2} \\$
$\text{Таким образом, параметры a и b не выражаются в явном виде через }\lambda$
$\text{и демонстрируют сложную нелинейную зависимость.}$ 
$\text{Дальнейшее аналитическое исследование задачи не целесообразно.}$




### Численное решение

In [3]:
q1= 1
q2= 2
p = -0.25

# Целевая функция
def objective(x):
    a, b = x
    return a**2 * q1**2 + b**2 * q2**2 + 2 * a * b * p * q1 * q2

# Ограничения
constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] - 1},
               {'type': 'ineq', 'fun': lambda x: x[0]},
               {'type': 'ineq', 'fun': lambda x: x[1]})

x0 = [0.5, 0.5]
result = minimize(objective, x0, constraints=constraints)
opt_a, opt_b = result.x
print(f'_w1: {round(opt_a,4)}, _w2: {round(opt_b,4)}, _minVar[S]: {result.fun}')


_w1: 0.75, _w2: 0.25, _minVar[S]: 0.625


## Задача 2 ##

Напишите алгоритм монте карло для симуляции цены
Используйте сгенерированные цены для нумерической оценки опционов используя Delta Hedging на каждом шаге цены (игнорируя комиссию)

Параметры Опциона Для оценки :
Strike = 1000
Implied Volatility = 0.3
Time to Expiration = Half a Year
interest rate = 0

Параметры для Монте Карло
Start Spot price = 1000
Realised Volatility = 0.2
Number of Paths = 50000
Number of Steps = N days untill the expiration

Напишите алгоритм нумерической апроксимации PNL данного опциона на момент экспирации. 
Финальное решение включающее в себя генерирование всех путей и оценку PNLa должно находиться в одном cell-e. На вывод должен быть принт финального пнла и тайминг экзекьюшена всего решения.

In [30]:
%%timeit
strike = 1000
ImpVol = 0.3
expiry = 0.5
rate = 0

spot = 1000
RealVol = 0.2
n_paths = 50000

def gbm(mu=0, rv=1, sims= 50000, T= 0.5, S0= 1000, cal=360):
    sims= int(sims)
    steps= int(cal * T)
    dt= T/steps
    St= np.exp((mu - 0.5 * rv ** 2) * dt + 
               rv * np.sqrt(dt) * mtalg.random.standard_normal(size=(sims, steps-1)).T)

    St = np.vstack([np.ones(sims), St])
    St = S0 * St.T.cumprod(axis=1)

    return St

def deltaGK(opt, S, K, iv, tau, sims, cal=360, rf=0, rd=0):
    steps= int(tau*cal)
    tau=np.linspace(start= 0, stop= tau, num=steps+1)[::-1][0:steps]
    tau=np.tile(tau, reps= sims).reshape(sims,steps)
    d1 = (np.log(S/K) + (rd - rf + (iv**2)/2)*tau)/(iv*np.sqrt(tau))
    delta= n_vect(d1)
    
    if opt == 'call':
        deltaCall= delta
        return deltaCall
    elif opt == 'put':
        deltaPut= delta - 1
        return deltaPut

def deltaHedge(cal=360):
    St= gbm(mu=0, rv=RealVol, sims= n_paths, T= expiry, S0= spot, cal=cal)

    deltas= deltaGK('call', S=St, K=strike, iv= ImpVol, sims=n_paths, \
        tau=expiry, rd=rate)

    dh_incr= np.diff(deltas)
    dh= np.vstack([deltas.T[0], dh_incr.T])
    dh= dh.T
    pnl_apx= np.sum((dh*St), axis=1)
    # return pnl_apx

    print(f'PnL Approximation: {round(np.mean(pnl_apx),2)} ± {round(np.std(pnl_apx),2)}')

deltaHedge()

PnL Approximation: 526.89 ± 472.52
PnL Approximation: 526.76 ± 472.44
PnL Approximation: 530.41 ± 472.83
PnL Approximation: 527.42 ± 472.61
PnL Approximation: 527.3 ± 472.24
PnL Approximation: 526.56 ± 472.31
PnL Approximation: 528.76 ± 472.87
PnL Approximation: 534.17 ± 473.19
607 ms ± 59.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Задача 3 ##

Даны случайные величины с нормальным распределением : X~N(mu,s), mu= 0 , s - ?
случайные величины эквивалентны дневному приросту цены.
Среднее абсолютное отклонение (Mean Average Deviation) (X) = 1%
чему равен s в годовом эквиваленте если мы закладываем 252 дня в году?


In [7]:
def extractAnnualVar(mad=0.01, cal=252):
    centeredSumX= mad*cal
    dailyVar= centeredSumX**2/(cal-1)
    annualisedVar= dailyVar*cal
    return annualisedVar

print(extractAnnualVar())

6.375700398406376
